# Tensorflow Algorithm

Material de Apoio: 

https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

In [ ]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(neurons, optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# modelo estratificado
kfold = StratifiedKFold(n_splits=10, shuffle=True)

# define the grid search parameters
neurons = [10, 15, 20, 25, 30, 35]
optimizer = ['SGD', 'RMSprop', 'Adam']
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(model__neurons=neurons, 
                  model__optimizer=optimizer, 
                  batch_size=batch_size, 
                  epochs=epochs)

grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-1, cv=kfold)

grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
